# A Simple Convolution Network

- Sobel operator in $x$ direction = 
$
    \begin{bmatrix} 
        -1 & 0 & 1 \\
        -2 & 0 & 2 \\
        -1 & 0 & 1
    \end{bmatrix}
$

- One (input, output) image tuple is given.
- http://docs.opencv.org/3.2.0/d2/d2c/tutorial_sobel_derivatives.html

In [1]:
import numpy as np
import tensorflow as tf
